In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from tabulate import tabulate  # New package for displaying results


In [2]:


# Load house price index data (assuming it exists as 'hpi_at_bdl_tract.csv')
url= 'https://www.fhfa.gov/hpi/download/annually/hpi_at_bdl_tract.csv'
hpi_data = pd.read_csv(url)


In [3]:
# Filter Data

hpi_dataFiltered = hpi_data[['tract', 'year', 'annual_change']]
hpi_dataFiltered.index = pd.to_datetime(hpi_dataFiltered['year']).dt.year
hpi_dataFiltered = hpi_dataFiltered[hpi_dataFiltered['annual_change'] != "."]  #remove specified missing data, NaN remains
hpi_dataFiltered['annual_change'] = hpi_dataFiltered['annual_change'].astype(float)

hpi_dataFiltered

,tract,year,annual_change
year,,,
1970,1001020100,1998,NaN
1970,1001020100,1999,-5.22
1970,1001020100,2000,4.77
1970,1001020100,2001,6.56
1970,1001020100,2002,8.59
...,...,...,...
1970,56045951300,2019,5.80
1970,56045951300,2020,1.70
1970,56045951300,2021,12.10


In [4]:
# Load cleaned tracts data
tracts = pd.read_csv('cleaned_tracts.csv')


In [5]:

# Merge datasets on GEOID (tract number)
df_uncleaned = tracts.merge(hpi_dataFiltered, left_on='GEOID', right_on='tract', how='left')

# Display first few rows to confirm merge
df_uncleaned.head()


,GEOID,STATEFP,COUNTYFP,metro_code,Hub_distance,NAME_2,NAMELSAD_2,Metro_Area,tract,year,annual_change
0,6029003305,6,29,37100.0,69691.301751,Los Angeles,Los Angeles County,Los Angeles,6.029003e+09,1986.0,NaN
1,6029003305,6,29,37100.0,69691.301751,Los Angeles,Los Angeles County,Los Angeles,6.029003e+09,1987.0,6.16
2,6029003305,6,29,37100.0,69691.301751,Los Angeles,Los Angeles County,Los Angeles,6.029003e+09,1988.0,8.74
3,6029003305,6,29,37100.0,69691.301751,Los Angeles,Los Angeles County,Los Angeles,6.029003e+09,1989.0,24.20
4,6029003305,6,29,37100.0,69691.301751,Los Angeles,Los Angeles County,Los Angeles,6.029003e+09,1990.0,12.41


In [6]:
# Check for missing values
df_uncleaned.isnull().sum()

#drop missing annual_change
df = df_uncleaned.dropna(subset=['annual_change'])

# Check for missing values
df.isnull().sum()


GEOID                0
STATEFP              0
COUNTYFP             0
metro_code       24931
Hub_distance         0
NAME_2               0
NAMELSAD_2           0
Metro_Area           0
tract                0
year                 0
annual_change        0
dtype: int64

In [17]:

# Store results
metro_results = []

# Loop through each metro area and run regression separately
for metro in df['Metro_Area'].unique():
    metro_df = df[df['Metro_Area'] == metro]  # Subset for the metro area

    if len(metro_df) < 30:  # Skip small metros with too few observations
        continue

    # Define variables
    X = sm.add_constant(metro_df['Hub_distance'])  # Add intercept
    y = metro_df['annual_change']

    # Fit regression model
    model = sm.OLS(y, X).fit()

    # Store results
    metro_results.append([
        metro,
        model.params['Hub_distance'],
        model.pvalues['Hub_distance'],
        model.rsquared
    ])

# Convert results to DataFrame
results_df = pd.DataFrame(metro_results, columns=['Metro_Area', 'Coef_Distance', 'P_Value', 'R_Squared'])

# Sort results by coefficient of Hub_distance
results_df = results_df.sort_values(by='Coef_Distance')

# Display results using tabulate
print("\nMetro Area Regression Results:")
print(tabulate(results_df, headers='keys', tablefmt='grid'))  # Nice tabulated format






Metro Area Regression Results:
+----+---------------+-----------------+------------+-------------+
|    | Metro_Area    |   Coef_Distance |    P_Value |   R_Squared |
+====+===============+=================+============+=============+
|  4 | Miami         |    -6.23177e-05 | 0.00125716 | 0.00088724  |
+----+---------------+-----------------+------------+-------------+
|  1 | San Diego     |    -2.16521e-05 | 0.00104924 | 0.000521851 |
+----+---------------+-----------------+------------+-------------+
|  0 | Los Angeles   |    -6.78346e-06 | 0.0150451  | 8.90472e-05 |
+----+---------------+-----------------+------------+-------------+
|  5 | Boston        |    -4.03872e-06 | 0.868246   | 8.66859e-06 |
+----+---------------+-----------------+------------+-------------+
|  3 | Jacksonville  |     9.09251e-07 | 0.946059   | 1.00021e-06 |
+----+---------------+-----------------+------------+-------------+
|  6 | New York      |     3.40114e-05 | 0.0674342  | 0.000235369 |
+----+----------

In [ ]:
# Key Takeaways for Investment Strategy
# ✅ Best Coastal Markets for Investment:

# Miami & San Diego have a strong negative coefficient, meaning closer-to-coast properties appreciate faster.
# Their p-values are very low (<0.05), confirming statistical significance.
# Investment Tip: Focus on coastal properties in Miami & San Diego, as they tend to gain value more than inland areas.

# ❌ Where Distance Does NOT Matter:

# Boston, Jacksonville, New York, San Francisco all have high p-values (>0.05), meaning distance does not affect price appreciation significantly.
# Investment Tip: Look at other factors (e.g., job growth, urban expansion) in these markets instead of just coastal proximity.
# 🚨 Surprising Result - New York!

# Positive coefficient (3.40e-05) suggests that inland areas appreciate more than coastal ones.
# However, p-value = 0.067 means it's not quite significant, so further research is needed.

In [ ]:
# my thoughts: this aligns with san diego and miami being beach tourist areas